# 使用 SageMaker 进行超参数自动调优

基于 SageMaker Python SDK 的超参数自动调优。目标是设定 1-3 个超参数，然后 SageMaker 会自动生成训练任务，使用不同的超参数组合。

## 升级 SageMaker Python SDK

SageMaker 有三个 SDK：

- Boto3 SDK，即 AWS SDK 的 Python 版，属于底层 SDK，未做封装，使用较为麻烦
- SageMaker Python SDK，即封装过的 SageMaker SDK，使用方便一些，但是需要查文档了解封装后的 API 与原服务 API 的对应关系
- SageMaker Experiements SDK，仅用于在 SageMaker Studio 中调用 Experiments 等功能

此处我们使用 SageMaker Python SDK。由于其已经升级到第 2 版，包含大量破坏性变更，而 SageMaker Studio 中的版本可能滞后，所以我们先进行更新。

In [ ]:
# 升级 SageMaker Python SDK v2
#
# [注] 此处换用清华 TUNA 镜像站加快速度

! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple --upgrade sagemaker==2.0.0

In [ ]:
# 项目参数
#
# - PROJECT = 项目名称，可自行修改或使用默认
# - BUCKET = S3 桶名字，用于存储图像数据，必须修改为自己的桶名字
# - CONTAINER_IMAGE = SageMaker 内置算法以封装好的容器形式存在，此为容器地址，请勿修改
#
# [注] 不同区域容器地址：https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

PROJECT = 'DEMO-IMAGE-CLSF'
BUCKET = 'sage-maker'

# 以下勿修改

CONTAINER_IMAGE = '387376663083.dkr.ecr.cn-northwest-1.amazonaws.com.cn/image-classification:1'

print('项目参数已设置。')

In [ ]:
# 常量超参数

NUM_LAYERS = 50
IMAGE_SHAPE = '3,224,224'
NUM_TRAINING_SAMPLES = 15420
NUM_CLASSES = 257
EPOCHS = 10

print('常量超参数已设置。')

In [ ]:
from time import time
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

# 初始化评估器

ROLE = get_execution_role()

ESTIMATOR = Estimator(
    image_uri = CONTAINER_IMAGE,
    role = ROLE, 
    instance_count = 1, 
    instance_type = 'ml.p3.8xlarge',
    volume_size = 50,
    max_run = 3600,
    use_spot_instances = True,
    max_wait = 7200, # 最长等待时间 = 最长运行时间 + 等待竞价实例时间，所以要 ≥ max_run
    input_mode = 'File',
    output_path = 's3://{}/{}/output'.format(BUCKET, PROJECT)
)

# 设置静态超参数

ESTIMATOR.set_hyperparameters(
    num_layers = NUM_LAYERS,
    image_shape = IMAGE_SHAPE,
    num_classes = NUM_CLASSES,
    num_training_samples = NUM_TRAINING_SAMPLES,
    epochs = EPOCHS,
    top_k = 2,
    augmentation_type = 'crop',
    # use_pretrained_model = 1  # SageMaker 内置算法的一键迁移学习功能
)

print('评估器已初始化。')

In [ ]:
# 查看优化结果

TUNER_METRICS = sagemaker.HyperparameterTuningJobAnalytics(TUNING_JOB_NAME)
TUNER_METRICS.dataframe().sort_values(
    ['FinalObjectiveValue'], 
    ascending = False
).head(5)

# 查看统计信息

# TODO

In [ ]:
# 获取最优评估器

try:
    BEST_ESTIMATOR = TUNER.best_estimator()
    
    print('已获取最优评估器')
except Exception as e:
    print('获取最优评估器时出错')
    raise e

# 部署最优评估器

BEST_ESTIMATOR.deploy()

## 热启动优化

热启动（warm start）优化指的是基于之前优化的结果进行进一步优化。有两种选择：

- **同构热启动优化**，即输入不变、算法不变、调优参数的选择不变，只变化调优参数的范围值
- **转移学习**，即输入可变、算法版本可变、调优参数的选择可变

### 同构热启动优化

In [ ]:
from time import time
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.inputs import TrainingInput

# 设置上游优化任务名字

PARENT_TUNING_JOB_NAME = 'DEMO-IMAGE-CLSF-1596601290'

# 可以再增加最多 4 个上游优化任务

ADDITIONAL_PARENT_TUNING_JOB_NAMES = {
    'DEMO-IMAGE-CLSF-1596709776'
}

# 连接已完成的优化任务

PARENT_TUNER = HyperparameterTuner.attach(
    tuning_job_name = PARENT_TUNING_JOB_NAME
)

PARENT_TUNER.estimator.max_wait = 7200 # !!!
PARENT_TUNER.estimator.use_spot_instances = True # !!!

PARENT_TUNER.estimator.set_hyperparameters(
    num_layers = 50,
    optimizer = 'rmsprop',
    epochs = 50
)

# 生成热启动优化器（同数据同算法）

IDENTICAL_TUNER = PARENT_TUNER.identical_dataset_and_algorithm_tuner(
    additional_parents = ADDITIONAL_PARENT_TUNING_JOB_NAMES
)

# 设置参数范围

HYPERPARAMS_RANGES = {
    'learning_rate': ContinuousParameter(0.00001, 0.01),
    'mini_batch_size': IntegerParameter(128, 256),
    'optimizer': CategoricalParameter(['sgd', 'adam', 'rmsprop', 'nag'])
}

IDENTICAL_TUNER._hyperparameter_ranges = HYPERPARAMS_RANGES # !!!

# 输入及验证数据地址（SDK Bug：不能自动继承输入）

S3_TRAIN_PATH = 's3://{}/{}/train/'.format(BUCKET, PROJECT)
S3_VALIDATION_PATH = 's3://{}/{}/validation/'.format(BUCKET, PROJECT)

TRAINING_DATA_SOURCE = TrainingInput(
    S3_TRAIN_PATH, 
    distribution = 'FullyReplicated', 
    content_type = 'application/x-recordio', 
    s3_data_type = 'S3Prefix'
)

VALIDATION_DATA_SOURCE = TrainingInput(
    S3_VALIDATION_PATH,
    distribution = 'FullyReplicated', 
    content_type = 'application/x-recordio', 
    s3_data_type = 'S3Prefix'
)

INPUTS = {
    'train': TRAINING_DATA_SOURCE, 
    'validation': VALIDATION_DATA_SOURCE
}

# 开始优化任务

TIMESTAMP = int(time())
TUNING_JOB_NAME = '{}-{}'.format(PROJECT, TIMESTAMP)

IDENTICAL_TUNER.fit(
    inputs = INPUTS,
    job_name = TUNING_JOB_NAME
)

IDENTICAL_TUNER.wait()

print('同构热启动优化任务完成。')

### 转移学习

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# TODO